# Code for running inference

10-18-2024

To convert this to a script, should have a name for the whole training task which goes in dataset.json and/or datalist.json because this will help for the task.json needed for inference.

Look into the minimal amount of metaata monai requires in the form of a task.json or other struct to initialize the builder and then run the ensemble


In [1]:
import os
from pathlib import Path
import numpy as np
import json
from loguru import logger
import platform

import mri_data
import monai_training

from monai.apps.auto3dseg import (
    AlgoEnsembleBestN,
    AlgoEnsembleBuilder,
    import_bundle_algo_history,
)
from monai.utils.enums import AlgoKeys

from reload_recursive import reload_recursive

/home/srs-9/.virtualenvs/monai/lib/python3.12/site-packages/ignite/handlers/checkpoint.py:17: DeprecationWarning: `TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.
  from torch.distributed.optim import ZeroRedundancyOptimizer


In [2]:
reload_recursive(mri_data)
reload_recursive(monai_training)
from mri_data.file_manager import scan_3Tpioneer_bids, DataSet, filter_first_ses  # noqa: E402, F401
from monai_training import preprocess  # noqa: E402


Setup logging


In [3]:
log_dir = ".logs"

if not os.path.exists(log_dir):
    os.makedirs(log_dir)
logger.add(
    os.path.join(log_dir, "file_{time:%Y_%m_%d}.log"), rotation="6h", level="DEBUG"
)

1

**Edit These**


In [4]:
#! Set these variables
work_dir_name = "choroid_pineal_pituitary3"
train_dataset_file_name = "training-dataset.json"
prediction_postfix = "choroid_pineal_pituitary3_pred"
task_name = "infer_choroid_pineal_pituitary"
modalities = ["flair", "t1"]

Set paths


In [5]:
hostname = platform.node()
if hostname == "rhinocampus" or hostname == "ryzen9":
    drive_root = Path("/media/smbshare")
else:
    drive_root = Path("/mnt/h")

projects_root = Path("/home/srs-9/Projects")

msmri_home = projects_root / "ms_mri"
training_work_dirs = msmri_home / "training_work_dirs"

# dataroot = "/media/hemondlab/Data/3Tpioneer_bids"
dataroot = drive_root / "3Tpioneer_bids"
work_dir = training_work_dirs / work_dir_name
train_dataset_file = work_dir / train_dataset_file_name
save_dir = drive_root / "3Tpioneer_bids_predictions"

prediction_filename = (
    ".".join(sorted(modalities)) + "_" + prediction_postfix + ".nii.gz"
)
print(prediction_filename)

taskfile_name = "inference-task.json"

flair.t1_choroid_pineal_pituitary3_pred.nii.gz


In [6]:
if not train_dataset_file.is_file():
    dataset = preprocess.parse_datalist(work_dir / "training-datalist.json", dataroot)
    preprocess.save_dataset(dataset, train_dataset_file)

In [7]:
def inference_exists(dataset: DataSet) -> DataSet:
    count = 0
    dataset_new = DataSet(dataset.dataroot)
    for scan in dataset:
        if not (save_dir / scan.relative_path / prediction_filename).is_file():
            dataset_new.append(scan)
        else:
            count += 1
    logger.info(f"{count} scans already have inference")
    return dataset_new

### Do Preparation

Get all the scans that were not in the training and testing set to create the inference dataset


In [14]:
# the scans that were used in the training
dataset_train, _ = preprocess.load_dataset(train_dataset_file)
dataset_train.dataroot = dataroot

# dataset_train2 has the same subject/sessions that are in dataset_train but with a subset of the keys
#   so that they can be compared to scans in the full data set when getting the set difference
dataset_proc = preprocess.DataSetProcesser.new_dataset(
    dataroot, scan_3Tpioneer_bids, filters=[filter_first_ses, inference_exists]
)
dataset_full = dataset_proc.dataset
dataset_train2 = DataSet.dataset_like(dataset_train, ["subid", "sesid"])
dataset_inference = DataSet.from_scans(set(dataset_full) - set(dataset_train2))

2024-12-31 09:50:34.208 | INFO     | __main__:inference_exists:9 - 308 scans already have inference


Prepare the images in the inference dataset (i.e combine flair and t1 images)


In [15]:
dataset_proc = preprocess.DataSetProcesser(dataset_inference)
dataset_proc.prepare_images(["flair", "t1"])
dataset_proc.dataset.sort(key=lambda s: s.subid)

2024-12-31 09:50:39.272 | INFO     | monai_training.preprocess:prepare_images:107 - Prepare Images
  0%|          | 0/227 [00:00<?, ?it/s]2024-12-31 09:50:39.451 | INFO     | mri_data.utils:merge_images:31 - fslmerge -a /mnt/h/3Tpioneer_bids/sub-ms2120/ses-20170920/flair.t1.nii.gz /mnt/h/3Tpioneer_bids/sub-ms2120/ses-20170920/flair.nii.gz /mnt/h/3Tpioneer_bids/sub-ms2120/ses-20170920/t1.nii.gz


Image Exception : #22 :: Failed to read volume /mnt/h/3Tpioneer_bids/sub-ms2120/ses-20170920/flair.nii.gz
Error : Error: short read, file may be truncated
terminate called after throwing an instance of 'std::runtime_error'
  what():  Failed to read volume /mnt/h/3Tpioneer_bids/sub-ms2120/ses-20170920/flair.nii.gz
Error : Error: short read, file may be truncated


2024-12-31 09:50:54.052 | ERROR    | monai_training.preprocess:prepare_images:147 - Something went wrong merging images for <bound method Scan.info of Scan(subid='2120', sesid='20170920', _dataroot=PosixPath('/mnt/h/3Tpioneer_bids'), _root=PosixPath('/mnt/h/3Tpioneer_bids/sub-ms2120/ses-20170920'), image=None, label=None, cond=None, id=42762350400)>
  5%|▍         | 11/227 [00:14<04:50,  1.34s/it]2024-12-31 09:50:54.082 | INFO     | mri_data.utils:merge_images:31 - fslmerge -a /mnt/h/3Tpioneer_bids/sub-ms1196/ses-20161004/flair.t1.nii.gz /mnt/h/3Tpioneer_bids/sub-ms1196/ses-20161004/flair.nii.gz /mnt/h/3Tpioneer_bids/sub-ms1196/ses-20161004/t1.nii.gz


Image Exception : #22 :: Failed to read volume /mnt/h/3Tpioneer_bids/sub-ms1196/ses-20161004/t1.nii.gz
Error : Error: short read, file may be truncated
terminate called after throwing an instance of 'std::runtime_error'
  what():  Failed to read volume /mnt/h/3Tpioneer_bids/sub-ms1196/ses-20161004/t1.nii.gz
Error : Error: short read, file may be truncated


2024-12-31 09:51:07.505 | ERROR    | monai_training.preprocess:prepare_images:147 - Something went wrong merging images for <bound method Scan.info of Scan(subid='1196', sesid='20161004', _dataroot=PosixPath('/mnt/h/3Tpioneer_bids'), _root=PosixPath('/mnt/h/3Tpioneer_bids/sub-ms1196/ses-20161004'), image=None, label=None, cond=None, id=24112560784)>
 12%|█▏        | 28/227 [00:28<03:11,  1.04it/s]2024-12-31 09:51:07.567 | WARNING  | monai_training.preprocess:prepare_images:138 - Couldn't prepare image for Scan(subid=1229, sesid=20170715)
2024-12-31 09:51:07.600 | WARNING  | monai_training.preprocess:prepare_images:138 - Couldn't prepare image for Scan(subid=1394, sesid=20191012)
2024-12-31 09:51:07.607 | WARNING  | monai_training.preprocess:prepare_images:138 - Couldn't prepare image for Scan(subid=1364, sesid=20170705)
 82%|████████▏ | 187/227 [00:28<00:03, 12.53it/s]2024-12-31 09:51:07.723 | WARNING  | monai_training.preprocess:prepare_images:138 - Couldn't prepare image for Scan(sub

Save the datalist and task files


In [16]:
images = []
for scan in dataset_proc.dataset:
    infile: Path = scan.image_path
    images.append({"image": str(infile.relative_to(dataset_proc.dataset.dataroot))})

logger.info(f"Will run inference on {len(images)} scans")

datalist = {"testing": images}

datalist_file = work_dir / "datalist.json"
with open(datalist_file, "w") as f:
    json.dump(datalist, f, indent=4)

task = {
    "name": task_name,
    "task": "segmentation",
    "modality": "MRI",
    "datalist": str(work_dir / "datalist.json"),
    "dataroot": str(dataroot),
}

task_file = os.path.join(work_dir, taskfile_name)
with open(task_file, "w") as f:
    json.dump(task, f, indent=4)

2024-12-31 09:54:14.370 | INFO     | __main__:<module>:6 - Will run inference on 217 scans


In [17]:
input_cfg = task_file  # path to the task input YAML file created by the users

history = import_bundle_algo_history(work_dir, only_trained=True)

In [19]:
## model ensemble
n_best = 5
builder = AlgoEnsembleBuilder(history, input_cfg)
builder.set_ensemble_method(AlgoEnsembleBestN(n_best=n_best))
ensemble = builder.get_ensemble()
save_params = {
    "_target_": "SaveImage",
    "output_dir": save_dir,
    "data_root_dir": dataroot,
    "output_postfix": prediction_postfix,
    "separate_folder": False,
}

pred = ensemble(pred_param={"image_save_func": save_params})

Ensembling (rank 0)...:   0%|          | 0/216 [00:00<?, ?it/s]You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast(

2024-12-31 09:58:13,565 INFO image_writer.py:197 - writing: /mnt/h/3Tpioneer_bids_predictions/sub-ms1465/ses-20160911/flair.t1_choroid_pineal_pituitary3_pred.nii.gz


Image save path not returned.
Ensembling (rank 0)...:   0%|          | 1/216 [03:08<11:13:46, 188.03s/it]You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
`torch.cuda.amp.autocast(args...)` is d

2024-12-31 10:01:47,573 INFO image_writer.py:197 - writing: /mnt/h/3Tpioneer_bids_predictions/sub-ms1467/ses-20220502/flair.t1_choroid_pineal_pituitary3_pred.nii.gz


Ensembling (rank 0)...:   1%|          | 2/216 [06:42<12:05:07, 203.31s/it]You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.a

...

2024-12-31 13:56:42,580 INFO image_writer.py:197 - writing: /mnt/h/3Tpioneer_bids_predictions/sub-ms2030/ses-20161112/flair.t1_choroid_pineal_pituitary3_pred.nii.gz


Ensembling (rank 0)...:  34%|███▍      | 74/216 [4:01:36<7:11:00, 182.12s/it]You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch

2024-12-31 14:00:00,289 INFO image_writer.py:197 - writing: /mnt/h/3Tpioneer_bids_predictions/sub-ms2033/ses-20170221/flair.t1_choroid_pineal_pituitary3_pred.nii.gz


Ensembling (rank 0)...:  35%|███▍      | 75/216 [4:04:54<7:19:00, 186.81s/it]You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
`torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch

KeyboardInterrupt: 